In [0]:
# Import Libraries

from pyspark.sql.window import Window
from pyspark.sql.functions import *
import urllib

In [0]:
# Mount S3 Bucket and Authentifications

# Read Delta table containing authentication credentials 
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"
aws_keys_df = spark.read.format("delta").load(delta_table_path)

# Get the AWS access key, secret key, and encode secret key.
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# S3 bucket information
AWS_S3_BUCKET = "user-126802f17de3-bucket"
MOUNT_NAME = "/mnt/databricks_s3_bucket"
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

# Unmount the previous mount to avoid conflicts, then mount the S3 bucket to Databricks
try:
    dbutils.fs.unmount(MOUNT_NAME)
except Exception:
    pass
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

/mnt/databricks_s3_bucket has been unmounted.
Out[418]: True

In [0]:
# Reading Data
  
# Reading and converting JSON into dataframes
def read_json_to_dataframe(file_location, file_type="json", infer_schema="true"):
    """
    Reads JSON data from the specified file location into a Spark DataFrame.

    Parameters:
    - file_location (str): The location of the JSON files.
    - file_type (str): The file type to read (default is "json").
    - infer_schema (str): Whether to infer the schema (default is "true").

    Returns:
    - pyspark.sql.DataFrame: The DataFrame containing the read JSON data.
    """
    spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")
    
    df = spark.read.format(file_type) \
        .option("inferSchema", infer_schema) \
        .load(file_location)

    return df

# Read data from S3 into a DataFrames
df_pin_uncleaned = read_json_to_dataframe("/mnt/databricks_s3_bucket/topics/126802f17de3.pin/partition=0/*.json")
df_geo_uncleaned = read_json_to_dataframe("/mnt/databricks_s3_bucket/topics/126802f17de3.geo/partition=0/*.json")
df_user_uncleaned = read_json_to_dataframe("/mnt/databricks_s3_bucket/topics/126802f17de3.user/partition=0/*.json")

# Uncomment the following lines if you want to display the DataFrames (uncleaned)
# display(df_pin_uncleaned)
# display(df_geo_uncleaned)
# display(df_user_uncleaned)

In [0]:
# Cleaning Pinterest DataFrame

def cleaning_pin(df):
    """
    Clean the pinterest data DataFrame (df_pin_uncleaned) by performing the following transformations:
    1. Replace empty entries and entries with no relevant data with None in specified columns.
    2. Transform follower_count to ensure every entry is a number and change its data type to int.
    3. Ensure that each column containing numeric data has a numeric data type.
    4. Clean the data in the save_location column to include only the save location path.
    5. Rename the index column to ind.
    6. Reorder the DataFrame columns.

    Parameters:
    - df (pyspark.sql.DataFrame): Input DataFrame to be cleaned.

    Returns:
    - pyspark.sql.DataFrame: Cleaned DataFrame.
    """

    null_dict = {
        "description": "No description available Story format",
        "follower_count": "User Info Error",
        "image_src": "Image src error.",
        "poster_name": "User Info Error",
        "tag_list": "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",
        "title": "No Title Data Available"
    }

    for key, value in null_dict.items():
        df = df.na.replace(value, None, key)

    df = df.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
    df = df.withColumn("follower_count", regexp_replace("follower_count", "M", "0000"))
    df = df.withColumn("follower_count", df["follower_count"].cast("int"))
    df = df.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))
    df = df.withColumnRenamed("index", "ind")
    df = df.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list",
                   "is_image_or_video", "image_src", "save_location", "category")
    return df

# Applying cleaning function
df_pin = cleaning_pin(df_pin_uncleaned)

In [0]:
# Cleaning Geolocation DataFrame

def cleaning_geo(df):
    """
    Clean the geolocation data DataFrame (df_geo_uncleaned) by performing the following transformations:
    1. Create a new column coordinates that contains an array based on the latitude and longitude columns.
    2. Drop the latitude and longitude columns from the DataFrame.
    3. Convert the timestamp column from a string to a timestamp data type.
    4. Reorder the DataFrame columns.

    Parameters:
    - df (pyspark.sql.DataFrame): Input DataFrame to be cleaned.

    Returns:
    - pyspark.sql.DataFrame: Cleaned DataFrame.
    """
    df = df.withColumn("coordinates", array("latitude", "longitude"))
    df = df.drop("latitude", "longitude")
    df = df.withColumn("timestamp", to_timestamp("timestamp"))
    df = df.select("ind", "country", "coordinates", "timestamp")
    return df

        
# Applying cleaning function
df_geo = cleaning_geo(df_geo_uncleaned)

In [0]:
# Cleaning User DataFrame

def cleaning_user(df):
    """
    Clean the user data DataFrame (df_user_uncleaned) by performing the following transformations:
    1. Create a new column user_name that concatenates the information found in the first_name and last_name columns.
    2. Drop the first_name and last_name columns from the DataFrame.
    3. Convert the date_joined column from a string to a timestamp data type.
    4. Reorder the DataFrame columns.

    Parameters:
    - df (pyspark.sql.DataFrame): Input DataFrame to be cleaned.

    Returns:
    - pyspark.sql.DataFrame: Cleaned DataFrame.
    """
    df = df.withColumn("user_name", concat("first_name", lit(" "), "last_name"))
    df = df.drop("first_name", "last_name")
    df = df.withColumn("date_joined", to_timestamp("date_joined"))
    df = df.select("ind", "user_name", "age", "date_joined")
    return df

        
# Applying cleaning function
df_user = cleaning_user(df_user_uncleaned)

In [0]:
# Query 1 - Most Popular Category in Each Country

# Join the df_pin DataFrame with the df_geo DataFrame using the "ind" column
df_pin_geo = df_pin.join(df_geo, "ind", "inner")

# Window specification for ranking within each country based on category_count
windowSpec_1 = Window.partitionBy("country").orderBy(col("category_count").desc())

# Count occurrences of each category within each country and rank them
query_1 = df_pin_geo.groupBy("country", "category") \
    .agg(count("category").alias("category_count")) \
    .withColumn("rank", rank().over(windowSpec_1)) \
    .filter(col("rank") == 1) \
    .drop("rank")

# Display the result
display(query_1)

country,category,category_count
Afghanistan,mens-fashion,3
Albania,art,10
Algeria,quotes,6
American Samoa,beauty,7
Andorra,quotes,3
Angola,diy-and-crafts,3
Anguilla,beauty,2
Anguilla,diy-and-crafts,2
Anguilla,home-decor,2
Antarctica (the territory South of 60 deg S),home-decor,1


In [0]:
# Query 2 - Most Popular Category Each Year (2018-2022)

# Perform grouping, aggregation, and ordering in a single step
query_2 = df_pin_geo.withColumn("post_year", year(col("timestamp"))) \
    .filter(col("post_year").between(2018, 2022)) \
    .groupBy("post_year", "category") \
    .agg(count("category").alias("category_count")) \
    .orderBy("post_year", desc("category_count"))

# Display the result
display(query_2)

post_year,category,category_count
2018,quotes,13
2018,christmas,11
2018,travel,10
2018,beauty,9
2018,art,7
2018,home-decor,7
2018,diy-and-crafts,7
2018,mens-fashion,6
2018,tattoos,6
2018,education,6


In [0]:
# Query 3 - (a) User with Most Followers in Each Country

# Step 1: Find the user with the most followers for each country
query_3a = df_pin_geo.groupBy("country", "poster_name") \
    .agg(max("follower_count").alias("follower_count")) \
    .orderBy(desc("follower_count"))

# Display the result
display(query_3a)


# Query 3 - (b) Country with User with Most Followers

# Step 2: Find the country with the user having the most followers
query_3b = query_3a.orderBy(col("follower_count").desc()) \
    .drop(col("poster_name")) \
    .limit(1)

# Display the result
display(query_3b)

country,poster_name,follower_count
Algeria,YourTango,942000
Lesotho,The Guardian,908000
Armenia,Michelle {CraftyMorning.com},892000
Aruba,GQ Magazine,874000
Trinidad and Tobago,Red Magazine,872000
Central African Republic,PureWow,868000
Argentina,Next Luxury,800000
Andorra,Glaminati,799000
Algeria,SheKnows,787000
Armenia,DIYnCrafts.Com,746000


country,follower_count
Algeria,942000


In [0]:
# Query 4 - Most Popular Category for Each Age Group

# Create a DataFrame and order by the custom order for age groups
df_pin_user = df_pin.join(df_user, "ind", "inner").withColumn(
    "age_group",
    when((col("age") >= 18) & (col("age") <= 24), "18-24")\
    .when((col("age") >= 25) & (col("age") <= 35), "25-35")\
    .when((col("age") >= 36) & (col("age") <= 50), "36-50")\
    .otherwise("+50")
)

# Define the custom order for age groups
age_group_order = expr("CASE WHEN age_group = '+50' THEN 1 ELSE 0 END"), col("age_group")

# Define window specification for partitioning age_group by category_count
windowSpec_2 = Window.partitionBy("age_group").orderBy(col("category_count").desc())

# Perform grouping, aggregation, and ordering in a single step
query_4 = df_pin_user.groupBy("age_group", "category") \
    .agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(windowSpec_2)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .orderBy(*age_group_order)

# Display the result
display(query_4)

age_group,category,category_count
18-24,tattoos,33
25-35,christmas,20
36-50,beauty,11
+50,beauty,7


In [0]:
# Query 5 - Median Follower Count for Different Age Groups

# Calculate the median follower count for each age group
query_5 = df_pin_user.groupBy("age_group") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .orderBy(*age_group_order)

# Display the result
display(query_5)

age_group,median_follower_count
18-24,50000
25-35,41000
36-50,5000
+50,560


In [0]:
# Query 6 - Number of Users Joined Each Year (2015-2020)

# Calculate the number of users who joined each year
query_6 = df_pin_user.withColumn("post_year", year(col("date_joined"))) \
    .filter(col("post_year").between(2015, 2020)) \
    .groupBy("post_year") \
    .agg(count("unique_id").alias("number_users_joined"))

# Display the result
display(query_6)

post_year,number_users_joined
2015,204
2016,228
2017,74


In [0]:
# Query 7 - Median Follower Count Based on Joining Year

# Calculate the median follower count for users who joined each year
query_7 = df_pin_user.withColumn("post_year", year(col("date_joined"))) \
    .filter(col("post_year").between(2015, 2020)) \
    .groupBy("post_year") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

# Display the result
display(query_7)

post_year,median_follower_count
2015,56000
2016,19000
2017,4000


In [0]:
# Query 8 - Median Follower Count Based on Joining Year and Age Group

# Calculate the median follower count for users who joined each year, based on age group
query_8 = df_pin_user.withColumn("post_year", year(col("date_joined"))) \
    .filter(col("post_year").between(2015, 2020)) \
    .groupBy("age_group", "post_year") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .orderBy(col("post_year").asc(), *age_group_order)

# Display the result
display(query_8)

age_group,post_year,median_follower_count
18-24,2015,91000
25-35,2015,63000
36-50,2015,10000
+50,2015,120
18-24,2016,23000
25-35,2016,40000
36-50,2016,9000
+50,2016,763
18-24,2017,10000
25-35,2017,6000
